In [1]:
import pandas as pd
# import fuzzywuzzy
# from fuzzywuzzy import process

# **Loading Data**

In [30]:
# 2021
df_laliga = pd.read_csv('content/laliga.csv')
df_bundesliga = pd.read_csv('content/1-bundesliga.csv')
df_liga1 = pd.read_csv('content/ligue-1.csv')
df_premier = pd.read_csv('content/premier-league.csv')
df_seria = pd.read_csv('content/serie-a.csv')

 # 2020
df_laliga_2020 = pd.read_csv('content/laliga_2020.csv')
df_bundesliga_2020 = pd.read_csv('content/1-bundesliga_2020.csv')
df_liga1_2020 = pd.read_csv('content/ligue-1_2020.csv')
df_premier_2020 = pd.read_csv('content/premier-league_2020.csv')
df_seria_2020 = pd.read_csv('content/serie-a_2020.csv')

# 2019
df_laliga_2019 = pd.read_csv('content/laliga_2019.csv')
df_bundesliga_2019 = pd.read_csv('content/1-bundesliga_2019.csv')
df_liga1_2019 = pd.read_csv('content/ligue-1_2019.csv')
df_premier_2019 = pd.read_csv('content/premier-league_2019.csv')
df_seria_2019 = pd.read_csv('content/serie-a_2019.csv')

# 2018
df_laliga_2018 = pd.read_csv('content/laliga_2018.csv')
df_bundesliga_2018 = pd.read_csv('content/1-bundesliga_2018.csv')
df_liga1_2018= pd.read_csv('content/ligue-1_2018.csv')
df_premier_2018 = pd.read_csv('content/premier-league_2018.csv')
df_seria_2018 = pd.read_csv('content/serie-a_2018.csv')

# 2017
df_laliga_2017 = pd.read_csv('content/laliga_2017.csv')
df_bundesliga_2017 = pd.read_csv('content/1-bundesliga_2017.csv')
df_liga1_2017= pd.read_csv('content/ligue-1_2017.csv')
df_premier_2017 = pd.read_csv('content/premier-league_2017.csv')
df_seria_2017 = pd.read_csv('content/serie-a_2017.csv')


# List of all DataFrames
dfs = [
    df_laliga, df_bundesliga, df_liga1, df_premier, df_seria,
    df_laliga_2020, df_bundesliga_2020, df_liga1_2020, df_premier_2020, df_seria_2020,
    df_laliga_2019, df_bundesliga_2019, df_liga1_2019, df_premier_2019, df_seria_2019,
    df_laliga_2018, df_bundesliga_2018, df_liga1_2018, df_premier_2018, df_seria_2018,
    df_laliga_2017, df_bundesliga_2017, df_liga1_2017, df_premier_2017, df_seria_2017
]

df_2021 = pd.concat([df_laliga, df_bundesliga, df_liga1, df_premier, df_seria],ignore_index=True)
df_2020 = pd.concat([ df_laliga_2020, df_bundesliga_2020, df_liga1_2020, df_premier_2020, df_seria_2020],ignore_index=True)
df_2019 = pd.concat([df_laliga_2019, df_bundesliga_2019, df_liga1_2019, df_premier_2019, df_seria_2019],ignore_index=True)
df_2018 = pd.concat([df_laliga_2018, df_bundesliga_2018, df_liga1_2018, df_premier_2018, df_seria_2018],ignore_index=True)
df_2017 = pd.concat([df_laliga_2017, df_bundesliga_2017, df_liga1_2017, df_premier_2017, df_seria_2017],ignore_index=True)

df_transfers_top5 = pd.read_csv('content/players_transfers__top5_leagues.csv')
# Concatenate all DataFrames
df = pd.concat(dfs, ignore_index=True)
df
df_transfers = [df_2021, df_2020, df_2019, df_2018, df_2017]

In [31]:
df_transfers_top5
df_transfers_top5.rename(columns={'price': 'fee'}, inplace=True)
df_transfers_top5['joined_club'] = pd.to_datetime(df_transfers_top5['joined_club'])


In [32]:
# # סינון הרשומות מתאריך 1 בינואר 2022 ואילך
df_transfers_top5= df_transfers_top5[df_transfers_top5['joined_club'] > '2022-01-01']
df_transfers_top5['year'] = df_transfers_top5['joined_club'].dt.year
df_transfers_top5 = df_transfers_top5[['name','age', 'height','nationality', 'fee', 'max_price', 'position', 'foot', 'year', 'league']]
df_transfers_top5 = df_transfers_top5[df_transfers_top5['league'] != 'Other']
df_transfers_top5_2023 = df_transfers_top5[df_transfers_top5['year'] == 2023]
df_transfers_top5_2022 = df_transfers_top5[df_transfers_top5['year'] == 2022]
#display(df_transfers_top5_2023)
#display(df_transfers_top5_2022)
df_2021.columns.tolist()

['club',
 'name',
 'age',
 'nationality',
 'position',
 'short_pos',
 'market_value',
 'dealing_club',
 'dealing_country',
 'fee',
 'movement',
 'window',
 'league',
 'season',
 'is_loan',
 'loan_status']

In [33]:
df_transfers_top5_2022.columns.tolist()
df_transfers_top5_2023 =  df_transfers_top5_2023[['name',
  'age',
 'fee',
 'position',
 'league']]

df_transfers_top5_2022= df_transfers_top5_2022[['name',
  'age',
 'fee',
 'position',
 'league']]


In [34]:
# פונקציה להוצאת השם מהמילה השנייה והלאה
def extract_last_name(full_name):
    name_parts = full_name.split(' ')
    if len(name_parts) > 1:
        return ' '.join(name_parts[1:])
    return full_name

# פונקציה להוצאת השם מהמילה השנייה והלאה
def extract_first_name(full_name):
    name_parts = full_name.split(' ')
    return ' '.join(name_parts[:1])



df = df_laliga.copy()
# יצירת עמודה חדשה עם השם מהמילה השנייה והלאה
df['last_name'] = df['name'].apply(extract_last_name)
df['first_name'] = df['name'].apply(extract_first_name)
df['full_name'] = df['first_name'] + ' ' + df['last_name']
df.head()

,club,name,age,nationality,position,short_pos,market_value,dealing_club,dealing_country,fee,movement,window,league,season,is_loan,loan_status,last_name,first_name,full_name
0,FC Barcelona,Rodrigo de Paul,27,Argentina,Central Midfield,CM,40000000.00,Udinese Calcio,Italy,35000000.00,in,summer,Laliga,2021,False,NaN,de Paul,Rodrigo,Rodrigo de Paul
1,FC Barcelona,Matheus Cunha,22,Brazil,Centre-Forward,CF,30000000.00,Hertha BSC,Germany,26000000.00,in,summer,Laliga,2021,False,NaN,Cunha,Matheus,Matheus Cunha
2,FC Barcelona,Antoine Griezmann,30,France,Second Striker,SS,50000000.00,Barcelona,Spain,10000000.00,in,summer,Laliga,2021,True,loan with fee,Griezmann,Antoine,Antoine Griezmann
3,FC Barcelona,Marcos Paulo,20,Portugal,Left Winger,LW,9000000.00,Fluminense,Brazil,0.00,in,summer,Laliga,2021,False,NaN,Paulo,Marcos,Marcos Paulo
4,FC Barcelona,Benjamin Lecomte,30,France,Goalkeeper,GK,9000000.00,Monaco,France,0.00,in,summer,Laliga,2021,True,free loan,Lecomte,Benjamin,Benjamin Lecomte


In [36]:
df_player_transfer = pd.read_csv('content/players_transfers__top5_leagues.csv')
df_player_transfer

,Unnamed: 0,name,full_name,age,height,nationality,place_of_birth,price,max_price,position,shirt_nr,foot,club,contract_expires,joined_club,player_agent,outfitter,league
0,1144,Enzo Barrenechea,Enzo Alan Tomás Barrenechea,21,1.86,Argentina,Villa María,0.90,0.90,midfield - Defensive Midfield,45,right,Juventus,6/30/2026,4/1/2023,SP,NaN,SerieA
1,1328,Marvin Zeegelaar,Marvin Romeo Kwasie Zeegelaar,32,1.86,Netherlands Suriname,Amsterdam,0.40,4.00,Defender - Left-Back,8,left,Udinese Calcio,6/30/2023,3/7/2023,NaN,NaN,SerieA
2,2374,Cheick Keita,NaN,20,1.85,France Mali,Champigny-sur-Marne,1.50,1.50,Defender - Centre-Back,43,right,Stade Reims,6/30/2026,3/1/2023,Nessim Zeggaie,NaN,Ligue1
3,1555,Marios Oikonomou,Μάριος Οικονόμου,30,1.89,Greece,Ioannina,0.30,2.50,Defender - Centre-Back,5,right,Sampdoria,6/30/2023,2/17/2023,SPORTS CONNECTION,NaN,SerieA
4,2085,Pape Cheikh,Pape Cheikh Diop Gueye,25,1.80,Senegal Spain,Guédiawaye,1.50,7.00,midfield - Central Midfield,12,right,Elche CF,6/30/2023,2/14/2023,4MB,Nike,LaLiga
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2607,1764,Iker Muniain,Iker Muniain Goñi,30,1.69,Spain,Pamplona,10.00,22.00,midfield - Attacking Midfield,10,right,Athletic,6/30/2024,7/1/2009,Jesus Medina Martin,NaN,LaLiga
2608,1828,Hugo Mallo,Hugo Mallo Novegil,31,1.74,Spain,Marín,3.50,15.00,Defender - Right-Back,2,right,Celta de Vigo,6/30/2023,7/1/2009,#LEADERS,NaN,LaLiga
2609,361,Seamus Coleman,Séamus Coleman,34,1.77,Ireland,Donegal,1.20,19.00,Defender - Right-Back,23,right,Everton,6/30/2023,1/1/2009,NaN,Nike,EPL
2610,1218,Stefan Radu,Ștefan Daniel Radu,36,1.83,Romania,Bukarest,0.40,10.00,Defender - Left-Back,26,left,Lazio,6/30/2023,7/1/2008,Matteo Materazzi,adidas,SerieA


In [37]:
def preprocess_transfers (df):
  # filtering
  df = df[(df['fee'] != 0) & (df['fee'].notna())]

  # filter out columns you don't need
  # columns_to_drop = ['loan_status', 'is_loan', 'dealing_country', 'dealing_club', 'movement', 'window', 'nationality']
  # df = df.drop(columns=columns_to_drop)

  # Counting Duplicates
  duplicate_count = df.duplicated().sum()
  #print(f"Number of duplicate rows: {duplicate_count}")

  duplicates = df[df.duplicated()]
  duplicates
  df = df[['name',
  'age',
 'fee',
 'position',
 'league']]
  return df

num_transfers = 0
for i in range (len(df_transfers)):
  print ("before", len(df_transfers[i]))
  df_transfers[i] = preprocess_transfers(df_transfers[i])
  num_transfers += len(df_transfers[i])
  print ("after", len(df_transfers[i]))

print ("total transfers", num_transfers)

print(len(df_transfers))
df_transfers_all = [df_transfers_top5_2023, df_transfers_top5_2022] + df_transfers
print(len(df_transfers_all))

before 2211
after 617
before 2784
after 783
before 3384
after 1211
before 3276
after 1219
before 3310
after 1194
total transfers 5024
5
7


In [38]:
for df in df_transfers_all:
  print (df.columns.tolist())

['name', 'age', 'fee', 'position', 'league']
['name', 'age', 'fee', 'position', 'league']
['name', 'age', 'fee', 'position', 'league']
['name', 'age', 'fee', 'position', 'league']
['name', 'age', 'fee', 'position', 'league']
['name', 'age', 'fee', 'position', 'league']
['name', 'age', 'fee', 'position', 'league']


In [39]:
pd.set_option('display.float_format', '{:.2f}'.format)
df.describe()


,age,fee
count,1194.00,1194.00
mean,24.63,8483988.27
std,3.70,16151706.42
min,16.00,1000.00
25%,22.00,1000000.00
50%,24.00,3000000.00
75%,27.00,9800000.00
max,36.00,222000000.00


### **Feature Extraction**
- need to decide which features to take from FIFA

In [40]:
# df_fifa_23 = pd.read_csv('/content/Fifa 23 Players Data.csv')
# df_fifa_22 = pd.read_csv('/content/players_22.csv')
# df_fifa_21 = pd.read_csv('/content/players_21.csv')
# df_fifa_20 = pd.read_csv('/content/players_20.csv')
# df_fifa_19 = pd.read_csv('/content/players_19.csv')
# df_fifa_18 = pd.read_csv('/content/players_18.csv')
# df_fifa_17 = pd.read_csv('/content/players_17.csv')

# df_fifas = [df_fifa_23, df_fifa_22, df_fifa_21, df_fifa_20, df_fifa_19 ,df_fifa_18 ,df_fifa_17]

# for i in range(len(df_fifas)):
#   print ("num rows in fifa before preprocess" , len(df_fifas[i]))


# # df_fifa.rename(columns={'short_name': 'name'}, inplace=True)

In [41]:
def pre_process_fifa (df):
  df['last_name'] = df['short_name'].apply(extract_last_name)
  df['first_name'] = df['long_name'].apply(extract_first_name)
  df['name'] = df['first_name'] + ' ' + df['last_name']
  standard_columns = {
      'nationality': 'nationality_name',
      'club': 'club_name',
      'defending_marking': 'defending_marking_awareness',
      'gk_diving': 'goalkeeping_diving',
      'gk_handling': 'goalkeeping_handling',
      'gk_kicking': 'goalkeeping_kicking',
      'gk_reflexes': 'goalkeeping_reflexes',
      'gk_speed': 'goalkeeping_speed',
      'gk_positioning': 'goalkeeping_positioning',
      'team_position': 'club_position',
      'team_jersey_number': 'club_jersey_number',
      'loaned_from': 'club_loaned_from',
      'joined': 'club_joined',
      'contract_valid_until': 'club_contract_valid_until',
      'age' : 'fifa_age'
  }

  df.rename(columns=standard_columns, inplace=True)

  columns_to_drop = ['player_url', 'dob', 'club_jersey_number', 'club_loaned_from', 'league_id' , 'fifa_age'
                       'body_type', 'nation_jersey_number', 'release_clause_eur', 'real_face', 'last_name',	'first_name',
                       'club_contract_valid_until', 'wage_eur', 'club_logo_url', 'club_flag_url', 'club_team_id', 'nationality_id', 'value_eur',
                      'player_id',  'fifa_update', 'update_as_of', 'short_name', 'long_name', 'club_team_id', 'club_joined_date', 'nation_team_id',
                       'nation_logo_url', 'nation_flag_url', 'league_name', 'league_rank', 'league_level', 'sofifa_id', 'player_face_url']
  columns_existing_to_drop = [col for col in columns_to_drop if col in df.columns]
  df = df.drop(columns=columns_existing_to_drop)
  return df




In [42]:
df_24 = pd.read_csv('content/Fifa_24.csv')
df_24 = df_24[df_24['fifa_version'] >= 17]
df_24 = pre_process_fifa (df_24)

C:\Users\רביד גרש\AppData\Local\Temp\ipykernel_39112\2907569030.py:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  df_24 = pd.read_csv('content/Fifa_24.csv')


In [212]:
full_joined_df = pd.DataFrame(columns = df_24.columns.tolist())
for i in range (len(df_transfers_all)):
    df_fifa_this_year = df_24[df_24['fifa_version'] ==(24 - i)]
    fifa_with_transfer = pd.merge(df_fifa_this_year, df_transfers_all[i], on='name', how='inner')
    fifa_with_transfer = fifa_with_transfer.drop_duplicates(subset=['name'])
    dfs_to_merge = [full_joined_df, fifa_with_transfer]
    full_joined_df = pd.concat(dfs_to_merge, ignore_index=True)
    print("iteration", i, full_joined_df['fifa_version'].unique())

print((full_joined_df.columns.tolist()))
display(full_joined_df)
print(len(full_joined_df))
full_joined_df.to_csv('merged_data.csv', index=False)




C:\Users\רביד גרש\AppData\Local\Temp\ipykernel_39112\1598889311.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_joined_df = pd.concat(dfs_to_merge, ignore_index=True)


iteration 0 [24.]
iteration 1 [24. 23.]
iteration 2 [24. 23. 22.]
iteration 3 [24. 23. 22. 21.]
iteration 4 [24. 23. 22. 21. 20.]
iteration 5 [24. 23. 22. 21. 20. 19.]
iteration 6 [24. 23. 22. 21. 20. 19. 18.]
['fifa_version', 'player_positions', 'overall', 'potential', 'fifa_age', 'height_cm', 'weight_kg', 'club_name', 'club_position', 'club_contract_valid_until_year', 'nationality_name', 'nation_position', 'preferred_foot', 'weak_foot', 'skill_moves', 'international_reputation', 'work_rate', 'body_type', 'player_tags', 'player_traits', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', '

,fifa_version,player_positions,overall,potential,fifa_age,height_cm,weight_kg,club_name,club_position,club_contract_valid_until_year,...,lcb,cb,rcb,rb,gk,name,age,fee,position,league
0,24.00,"LB, RB",86,86,29,182,74,FC Barcelona,RB,2024.00,...,78+3,78+3,78+3,83+3,19+3,João Cancelo,28.00,60.00,Defender - Right-Back,Bundesliga
1,24.00,GK,84,84,34,183,79,Inter,GK,2026.00,...,29+3,29+3,29+3,29+3,83+1,Yann Sommer,34.00,5.00,Goalkeeper,Bundesliga
2,24.00,"CF, LW, ST",84,84,29,176,78,Atlético Madrid,SUB,2025.00,...,50+3,50+3,50+3,56+3,19+3,Memphis Depay,29.00,16.00,Attack - Centre-Forward,LaLiga
3,24.00,"CM, CDM",83,89,22,178,76,Chelsea,CAM,2031.00,...,76+3,76+3,76+3,76+3,16+3,Enzo Fernández,22.00,85.00,midfield - Defensive Midfield,EPL
4,24.00,"CF, LW",83,88,24,189,76,Liverpool,CF,2028.00,...,55+2,55+2,55+2,61+2,18+2,Cody Gakpo,24.00,60.00,Attack - Left Winger,EPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3261,18.00,"LB, LM",60,67,22,178,74,Cesena,LM,2019.00,...,58+1,58+1,58+1,59+1,18+1,Fabio Eguelfi,22.00,6000000.00,Left-Back,Serie A
3262,18.00,LB,60,63,25,170,64,Cittadella,SUB,2020.00,...,60+1,60+1,60+1,59+1,16+1,Amedeo Benedetti,25.00,10000.00,Left-Back,Serie A
3263,18.00,CM,59,75,20,183,75,Spezia,SUB,2022.00,...,53+1,53+1,53+1,52+1,15+1,Matteo Pessina,20.00,1650000.00,Attacking Midfield,Serie A
3264,18.00,RM,56,72,19,189,83,Udinese,RES,2021.00,...,35+1,35+1,35+1,40+1,16+1,Svante Ingelsson,19.00,750000.00,Central Midfield,Serie A


3266


[דברים לבדוק
נרמול שנים
נרמול שנת סיום חוזה לפי אורך חזה
למחוק ציון לפי כל עמדה
לעשות one hot code לכל עמדה ולשמור מחיר עמדה](https:// [link text](https://))

change positions col to 
1 - gk
2- defence
3- midfield
4- attack
#TODO so we can abandon the player_position and club_position columns

In [219]:
full_joined_df['position'].unique()

array([77, 54, 83, 70, 84, 85, 80, 88, 75, 51, 65, 55, 68, 57, 63, 67, 69,
       74, 58, 62, 93, 73, 34, 42, 66, 60, 90, 64, 40, 82, 71, 72, 78, 81,
       59, 76, 52, 43, 50, 35, 36, 91, 53, 87, 44, 61, 49, 86, 92, 39, 45,
       48, 31, 94, 79, 89, 56, 47, 18, 33, 95, 41, 38, 28, 27, 37, 25, 21,
       32, 24, 17, 22, 20, 16, 26, 46, 30, 96], dtype=object)

15


,fifa_version,player_positions,overall,potential,fifa_age,height_cm,weight_kg,club_name,club_position,club_contract_valid_until_year,...,lcb,cb,rcb,rb,gk,name,age,fee,position,league
0,24.00,LB,86,86,29,182,74,FC Barcelona,RB,2024.00,...,78+3,78+3,78+3,83+3,19+3,João Cancelo,28.00,60.00,Defender - Right-Back,Bundesliga
1,24.00,GK,84,84,34,183,79,Inter,GK,2026.00,...,29+3,29+3,29+3,29+3,83+1,Yann Sommer,34.00,5.00,Goalkeeper,Bundesliga
2,24.00,CF,84,84,29,176,78,Atlético Madrid,SUB,2025.00,...,50+3,50+3,50+3,56+3,19+3,Memphis Depay,29.00,16.00,Attack - Centre-Forward,LaLiga
3,24.00,CM,83,89,22,178,76,Chelsea,CAM,2031.00,...,76+3,76+3,76+3,76+3,16+3,Enzo Fernández,22.00,85.00,midfield - Defensive Midfield,EPL
4,24.00,CF,83,88,24,189,76,Liverpool,CF,2028.00,...,55+2,55+2,55+2,61+2,18+2,Cody Gakpo,24.00,60.00,Attack - Left Winger,EPL


In [214]:
mapping ={
'Defender - Right-Back': 2,
'Goalkeeper': 1,
'Attack - Centre-Forward': 4,
'midfield - Defensive Midfield': 3,
'Attack - Left Winger': 4,
'Attack - Right Winger': 4,
'Attack - Second Striker': 4,
'midfield - Central Midfield':3,
'Defender - Centre-Back': 2,
'midfield - Attacking Midfield': 3,
'Defender - Left-Back': 2,
'midfield - Right Midfield': 3,
'midfield - Left Midfield': 3,
'Centre-Forward': 4,
'Left Winger': 4,
'Centre-Back': 2,
'Right-Back': 2,
'Second Striker': 4,
'Central Midfield': 3,
'Attacking Midfield': 3,
'Left Midfield': 3,
'Defensive Midfield': 2,
'Right Winger': 4,
'Right Midfield': 3,
 'Left-Back': 2

}

full_joined_df['position'].head()
# מיפוי הערך בעמודה לשורה אחת
full_joined_df['position'] = full_joined_df['position'].map(mapping)
full_joined_df['position'].unique()

array([2, 1, 4, 3], dtype=int64)

In [210]:
pos_list = ['','GK', 'DEF', 'MID', 'ATT']
for i in range(1,5):
    print(f"number of players in position {pos_list[i]}", len(full_joined_df[full_joined_df['position'] == i]))

number of players in position GK 227
number of players in position DEF 1252
number of players in position MID 719
number of players in position ATT 1068


player_traits and player_tags represent special skills
we will count them for each player

In [211]:
# player tags represent the player skills
full_joined_df['player_tags'].unique()
full_joined_df['player_traits'].unique()

array([1, 0, 2, 4, 3, 5, 7, 6], dtype=int64)

In [216]:
#if the player didnt have skill is appear as nan so then we will define it as 0
#the skills separate by , so we will split them and then count
full_joined_df['player_tags'] = full_joined_df['player_tags'].apply(lambda x: len(set(x.split(','))) if isinstance(x, str) else 0)
full_joined_df['player_traits'] = full_joined_df['player_traits'].apply(lambda x: len(set(x.split(','))) if isinstance(x, str) else 0)



In [217]:
full_joined_df['player_tags'].unique()
full_joined_df['player_traits'].unique()

array([1, 0, 2, 4, 3, 5, 7, 6], dtype=int64)

מחוץ ל5 הליגות הבכירות
הולנד
בלגיה
רוסיה
פורטוגל
סקוטלנד

In [141]:
# 2021
df_holland = pd.read_csv('content/eredivisie.csv')
df_belgium = pd.read_csv('content/jupiler-pro-league.csv')
df_russia = pd.read_csv('content/premier-liga.csv')
df_portugal = pd.read_csv('content/primeira-liga.csv')
df_scotland = pd.read_csv('content/scottish-premiership.csv')

 # 2020
df_holland_2020 = pd.read_csv('content/eredivisie_2020.csv')
df_belgium_2020 = pd.read_csv('content/jupiler-pro-league_2020.csv')
df_russia_2020 = pd.read_csv('content/premier-liga_2020.csv')
df_portugal_2020 = pd.read_csv('content/primeira-liga_2020.csv')
df_scotland_2020 = pd.read_csv('content/scottish-premiership_2020.csv')

# 2019
df_holland_2019 = pd.read_csv('content/eredivisie_2019.csv')
df_belgium_2019 = pd.read_csv('content/jupiler-pro-league_2019.csv')
df_russia_2019 = pd.read_csv('content/premier-liga_2019.csv')
df_portugal_2019 = pd.read_csv('content/primeira-liga_2019.csv')
df_scotland_2019 = pd.read_csv('content/scottish-premiership_2019.csv')

# 2018
df_holland_2018 = pd.read_csv('content/eredivisie_2018.csv')
df_belgium_2018 = pd.read_csv('content/jupiler-pro-league_2018.csv')
df_russia_2018 = pd.read_csv('content/premier-liga_2018.csv')
df_portugal_2018 = pd.read_csv('content/primeira-liga_2018.csv')
df_scotland_2018 = pd.read_csv('content/scottish-premiership_2018.csv')

# 2017
df_holland_2017 = pd.read_csv('content/eredivisie_2017.csv')
df_belgium_2017 = pd.read_csv('content/jupiler-pro-league_2017.csv')
df_russia_2017 = pd.read_csv('content/premier-liga_2017.csv')
df_portugal_2017 = pd.read_csv('content/primeira-liga_2017.csv')
df_scotland_2017 = pd.read_csv('content/scottish-premiership_2017.csv')


# List of all DataFrames
dfs = [
    df_holland, df_belgium, df_russia, df_portugal, df_scotland,
    df_holland_2020, df_belgium_2020, df_russia_2020, df_portugal_2020, df_scotland_2020,
    df_holland_2019, df_belgium_2019, df_russia_2019, df_portugal_2019, df_scotland_2019,
    df_holland_2018, df_belgium_2018, df_russia_2018, df_portugal_2018, df_scotland_2018,
    df_holland_2017, df_belgium_2017, df_russia_2017, df_portugal_2017, df_scotland_2017
]

df_2021 = pd.concat([df_holland, df_belgium, df_russia, df_portugal, df_scotland],ignore_index=True)
df_2020 = pd.concat([df_holland_2020, df_belgium_2020, df_russia_2020, df_portugal_2020, df_scotland_2020],ignore_index=True)
df_2019 = pd.concat([df_holland_2019, df_belgium_2019, df_russia_2019, df_portugal_2019, df_scotland_2019],ignore_index=True)
df_2018 = pd.concat([df_holland_2018, df_belgium_2018, df_russia_2018, df_portugal_2018, df_scotland_2018],ignore_index=True)
df_2017 = pd.concat([df_holland_2017, df_belgium_2017, df_russia_2017, df_portugal_2017, df_scotland_2017],ignore_index=True)

df_transfers_top5 = pd.read_csv('content/players_transfers__top5_leagues.csv')
# Concatenate all DataFrames
df = pd.concat(dfs, ignore_index=True)
df.head()
df_transfers_out_of_top_5 = [df_2021, df_2020, df_2019, df_2018, df_2017]

In [142]:
num_transfers = 0
for i in range (len(df_transfers_out_of_top_5)):
  print ("before", len(df_transfers_out_of_top_5[i]))
  df_transfers_out_of_top_5[i] = preprocess_transfers(df_transfers_out_of_top_5[i])
  num_transfers += len(df_transfers_out_of_top_5[i])
  print ("after", len(df_transfers_out_of_top_5[i]))

print ("total transfers", num_transfers)



before 1772
after 290
before 2609
after 381
before 2635
after 475
before 2631
after 452
before 2779
after 469
total transfers 2067


In [143]:
full_joined_df_out_of_top_5 = pd.DataFrame(columns = df_24.columns.tolist())
for i in range (len(df_transfers_out_of_top_5)):
    df_fifa_this_year = df_24[df_24['fifa_version'] ==(24 - i)]
    fifa_with_transfer = pd.merge(df_fifa_this_year, df_transfers_out_of_top_5[i], on='name', how='inner')
    fifa_with_transfer = fifa_with_transfer.drop_duplicates(subset=['name'])
    dfs_to_merge = [full_joined_df_out_of_top_5, fifa_with_transfer]
    full_joined_df_out_of_top_5 = pd.concat(dfs_to_merge, ignore_index=True)
    print("iteration", i, full_joined_df_out_of_top_5['fifa_version'].unique(), len(full_joined_df_out_of_top_5))

print((full_joined_df_out_of_top_5.columns.tolist()))
display(full_joined_df_out_of_top_5)
print(len(full_joined_df_out_of_top_5))
full_joined_df_out_of_top_5.to_csv('merged_data.csv', index=False)

full_joined_df_out_of_top_5.head()


C:\Users\רביד גרש\AppData\Local\Temp\ipykernel_39112\556064823.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_joined_df_out_of_top_5 = pd.concat(dfs_to_merge, ignore_index=True)


iteration 0 [24.] 133
iteration 1 [24. 23.] 284
iteration 2 [24. 23. 22.] 500
iteration 3 [24. 23. 22. 21.] 695
iteration 4 [24. 23. 22. 21. 20.] 913
['fifa_version', 'player_positions', 'overall', 'potential', 'fifa_age', 'height_cm', 'weight_kg', 'club_name', 'club_position', 'club_contract_valid_until_year', 'nationality_name', 'nation_position', 'preferred_foot', 'weak_foot', 'skill_moves', 'international_reputation', 'work_rate', 'body_type', 'player_tags', 'player_traits', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 'mentality_aggression',

,fifa_version,player_positions,overall,potential,fifa_age,height_cm,weight_kg,club_name,club_position,club_contract_valid_until_year,...,lcb,cb,rcb,rb,gk,name,age,fee,position,league
0,24.00,"LB, LWB",82,87,21,176,70,Paris Saint Germain,SUB,2026.00,...,77+2,77+2,77+2,80+2,18+2,Nuno Mendes,19.00,7000000.00,Left-Back,Primeira Liga
1,24.00,"RM, ST",82,84,24,178,77,Borussia Dortmund,RM,2026.00,...,51+2,51+2,51+2,59+2,18+2,Donyell Malen,22.00,30000000.00,Centre-Forward,Eredivisie
2,24.00,"CDM, CM",81,86,22,182,77,Paris Saint Germain,CDM,2028.00,...,76+2,76+2,76+2,76+2,17+2,Manuel Ugarte,20.00,6500000.00,Defensive Midfield,Primeira Liga
3,24.00,"RWB, RM",81,82,27,188,80,Inter,RM,2025.00,...,80+2,80+2,80+2,80+2,19+3,Denzel Dumfries,25.00,12500000.00,Right-Back,Eredivisie
4,24.00,"CAM, CM, RW",81,81,31,182,75,Ajax,RW,2025.00,...,51+3,51+3,51+3,59+3,18+3,Steven Berghuis,29.00,5500000.00,Right Winger,Eredivisie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
908,20.00,CB,65,69,25,191,83,Livorno,LCB,2024.00,...,64+2,64+2,64+2,57+2,18+2,Matija Boben,23.00,300000.00,Centre-Back,Premier Liga
909,20.00,LW,65,68,25,185,68,Grenoble Foot 38,SUB,2021.00,...,46+2,46+2,46+2,51+2,17+2,Willy Semedo,23.00,130000.00,Left Winger,Jupiler Pro League
910,20.00,CM,64,67,26,185,73,Odd,CM,2019.00,...,61+2,61+2,61+2,60+2,18+2,Fredrik Jensen,24.00,650000.00,Defensive Midfield,Jupiler Pro League
911,20.00,"CB, CDM",60,73,19,187,82,Royal Excel Mouscron,SUB,2022.00,...,59+2,59+2,59+2,53+2,15+2,Rijad Sadiku,18.00,300000.00,Centre-Back,Jupiler Pro League


913


,fifa_version,player_positions,overall,potential,fifa_age,height_cm,weight_kg,club_name,club_position,club_contract_valid_until_year,...,lcb,cb,rcb,rb,gk,name,age,fee,position,league
0,24.00,"LB, LWB",82,87,21,176,70,Paris Saint Germain,SUB,2026.00,...,77+2,77+2,77+2,80+2,18+2,Nuno Mendes,19.00,7000000.00,Left-Back,Primeira Liga
1,24.00,"RM, ST",82,84,24,178,77,Borussia Dortmund,RM,2026.00,...,51+2,51+2,51+2,59+2,18+2,Donyell Malen,22.00,30000000.00,Centre-Forward,Eredivisie
2,24.00,"CDM, CM",81,86,22,182,77,Paris Saint Germain,CDM,2028.00,...,76+2,76+2,76+2,76+2,17+2,Manuel Ugarte,20.00,6500000.00,Defensive Midfield,Primeira Liga
3,24.00,"RWB, RM",81,82,27,188,80,Inter,RM,2025.00,...,80+2,80+2,80+2,80+2,19+3,Denzel Dumfries,25.00,12500000.00,Right-Back,Eredivisie
4,24.00,"CAM, CM, RW",81,81,31,182,75,Ajax,RW,2025.00,...,51+3,51+3,51+3,59+3,18+3,Steven Berghuis,29.00,5500000.00,Right Winger,Eredivisie


In [ ]:
# def pre_process_fifa (df, year):
#   df['last_name'] = df['short_name'].apply(extract_last_name)
#   df['first_name'] = df['long_name'].apply(extract_first_name)
#   df['full_name'] = df['first_name'] + ' ' + df['last_name']
#   # Standardize column names
#   standard_columns = {
#       'nationality': 'nationality_name',
#       'club': 'club_name',
#       'defending_marking': 'defending_marking_awareness',
#       'gk_diving': 'goalkeeping_diving',
#       'gk_handling': 'goalkeeping_handling',
#       'gk_kicking': 'goalkeeping_kicking',
#       'gk_reflexes': 'goalkeeping_reflexes',
#       'gk_speed': 'goalkeeping_speed',
#       'gk_positioning': 'goalkeeping_positioning',
#       'team_position': 'club_position',
#       'team_jersey_number': 'club_jersey_number',
#       'loaned_from': 'club_loaned_from',
#       'joined': 'club_joined',
#       'contract_valid_until': 'club_contract_valid_until'
#   }

#   df.rename(columns=standard_columns, inplace=True)

# # TODO WITHOUT CLUB IN TOP 5
#   # df= df[df['club_name'].isin(clubs_top_5)]
#   columns_to_drop = ['player_url', 'dob', 'club_jersey_number', 'club_loaned_from',
#                        'body_type', 'nation_jersey_number', 'release_clause_eur', 'real_face',
#                        'club_contract_valid_until', 'wage_eur', 'club_logo_url', 'club_flag_url',
#                        'nation_logo_url', 'nation_flag_url', 'league_name', 'league_rank', 'league_level', 'sofifa_id', 'player_face_url']

#   columns_existing_to_drop = [col for col in columns_to_drop if col in df.columns]
#   df = df.drop(columns=columns_existing_to_drop)


#   # TODO WITHOUT RECENTLY JOIN

#   # df['club_joined'] = pd.to_datetime(df['club_joined'])

#   # df = df[df['club_joined'].dt.year >= year]


#   df.head()
#   return df



# clubs_22 = set((df_fifa_22[df_fifa_22['league_name'].isin
# (['Italian Serie A', 'German 1. Bundesliga', 'French Ligue 1', 'English Premier League', 'Spain Primera Division' ])]['club_name'].unique()))

# clubs_21 = set((df_fifa_21[df_fifa_21['league_name'].isin
# (['Italian Serie A', 'German 1. Bundesliga', 'French Ligue 1', 'English Premier League', 'Spain Primera Division' ])]['club_name'].unique()))

# clubs_top_5 = list(clubs_21 | clubs_22)

# for i in range (len(df_fifas)):
#   if i == 0:
#     continue
#   print ("num rows in fifa before preprocess" , len(df_fifas[i]))
#   df_fifas[i] = pre_process_fifa(df_fifas[i], 2023-1-i )
#   # print(df_fifas[i]['club_joined'].dt.year.unique())
#   print ("num rows in fifa after preprocess" , len(df_fifas[i]))
#   print("num cols in fifa", len(df_fifas[i].columns.tolist()))




num rows in fifa before preprocess 16230
num rows in fifa after preprocess 16230
num cols in fifa 95
num rows in fifa before preprocess 18944
num rows in fifa after preprocess 18944
num cols in fifa 96
num rows in fifa before preprocess 18278
num rows in fifa after preprocess 18278
num cols in fifa 96
num rows in fifa before preprocess 17770
num rows in fifa after preprocess 17770
num cols in fifa 96
num rows in fifa before preprocess 17592
num rows in fifa after preprocess 17592
num cols in fifa 96
num rows in fifa before preprocess 17009
num rows in fifa after preprocess 17009
num cols in fifa 96


In [ ]:
# for i in range(len(df_fifas)):
#     # יצירת שם קובץ לכל DataFrame על פי השנה המתאימה
#     filename = f'fifa_{2023-i}_try.csv'
#     # שמירת ה-DataFrame לקובץ CSV
#     df_fifas[i].to_csv(filename, index=False)

In [ ]:
# for i in range(len(df_transfers_all)):
#     # יצירת שם קובץ לכל DataFrame על פי השנה המתאימה
#     filename = f'transfers_{2023-i}_try.csv'
#     # שמירת ה-DataFrame לקובץ CSV
#     df_transfers_all[i].to_csv(filename, index=False)

In [ ]:
# df_fifas = df_fifas[:-1]
# d
# # # ג'וין חוץ מ2023
# for i in range (len(df_fifas)):
#   if i == 0: continue


#   # ביצוע ה-merge
#   merged_df = pd.merge(df_fifas[i], df_transfers_all[i+1], left_on='full_name', right_on='name', how='inner')
#   merged_df = merged_df.drop_duplicates(subset=['full_name'])
#   display(merged_df)
#   print(len(merged_df))



,short_name,long_name,player_positions,overall,potential,value_eur,age_x,height_cm,weight_kg,club_team_id,...,rb,gk,last_name,first_name,full_name,name,age_y,fee,position,league
0,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.00,36,187,83,11.00,...,60+3,20+3,Ronaldo,Cristiano,Cristiano Ronaldo,Cristiano Ronaldo,36,15000000.00,Centre-Forward,Premier League
2,R. Lukaku,Romelu Lukaku Menama,ST,88,88,93500000.00,28,191,94,5.00,...,58+3,19+3,Lukaku,Romelu,Romelu Lukaku,Romelu Lukaku,28,115000000.00,Centre-Forward,Premier League
4,J. Sancho,Jadon Sancho,"RM, CF, LM",87,91,116500000.00,21,180,76,11.00,...,60+3,19+3,Sancho,Jadon,Jadon Sancho,Jadon Sancho,21,85000000.00,Left Winger,1 Bundesliga
6,R. Varane,Raphaël Varane,CB,86,88,68500000.00,28,191,81,11.00,...,80+3,18+3,Varane,Raphaël,Raphaël Varane,Raphaël Varane,28,40000000.00,Centre-Back,Laliga
8,A. Griezmann,Antoine Griezmann,"ST, LW, RW",85,85,53000000.00,30,176,73,240.00,...,69+3,21+3,Griezmann,Antoine,Antoine Griezmann,Antoine Griezmann,30,10000000.00,Second Striker,Laliga
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,T. Siersleben,Tim Siersleben,"CB, LB",63,75,1100000.00,21,187,80,111235.00,...,60+2,16+2,Siersleben,Tim,Tim Siersleben,Tim Siersleben,21,150000.00,Centre-Back,1 Bundesliga
453,L. Tempelmann,Lino Tempelmann,CM,62,71,900000.00,22,174,77,171.00,...,59+2,14+2,Tempelmann,Lino,Lino Tempelmann,Lino Tempelmann,22,50000.00,Central Midfield,1 Bundesliga
454,D. Ramaj,Diant Ramaj,GK,62,77,825000.00,19,186,80,1824.00,...,17+2,61+2,Ramaj,Diant,Diant Ramaj,Diant Ramaj,19,100000.00,Goalkeeper,1 Bundesliga
455,A. Sher,Aimar Sher,CM,62,77,925000.00,18,175,65,110741.00,...,56+2,15+2,Sher,Aimar,Aimar Sher,Aimar Sher,18,2400000.00,Central Midfield,Serie A


317


,short_name,long_name,age_x,height_cm,weight_kg,nationality_name,club_name,overall,potential,value_eur,...,rcb,rb,last_name,first_name,full_name,name,age_y,fee,position,league
0,L. Suárez,Luis Alberto Suárez Díaz,33,182,86,Uruguay,FC Barcelona,87,87,31500000,...,63+3,64+3,Suárez,Luis,Luis Suárez,Luis Suárez,33,7000000.00,Centre-Forward,Laliga
6,K. Havertz,Kai Havertz,21,188,83,Germany,Chelsea,85,93,57000000,...,56+3,63+3,Havertz,Kai,Kai Havertz,Kai Havertz,21,80000000.00,Attacking Midfield,1 Bundesliga
8,L. Sané,Leroy Sané,24,183,75,Germany,FC Bayern München,85,90,51500000,...,52+2,60+2,Sané,Leroy,Leroy Sané,Leroy Sané,24,45000000.00,Right Winger,1 Bundesliga
10,T. Werner,Timo Werner,24,180,76,Germany,Chelsea,85,88,49500000,...,50+3,58+3,Werner,Timo,Timo Werner,Timo Werner,24,53000000.00,Centre-Forward,1 Bundesliga
12,M. Icardi,Mauro Emanuel Icardi Rivero,27,181,75,Argentina,Paris Saint-Germain,85,86,45000000,...,51+3,51+3,Icardi,Mauro,Mauro Icardi,Mauro Icardi,27,50000000.00,Centre-Forward,Ligue 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,N. Rovella,Nicolò Rovella,18,179,70,Italy,Genoa,58,77,250000,...,53+2,55+2,Rovella,Nicolò,Nicolò Rovella,Nicolò Rovella,19,18000000.00,Defensive Midfield,Serie A
582,L. Rojas,Luis Rojas,18,180,82,Chile,Crotone,57,79,230000,...,41+2,46+2,Rojas,Luis,Luis Rojas,Luis Rojas,18,2400000.00,Attacking Midfield,Serie A
583,K. Askildsen,Kristoffer Askildsen,19,190,77,Norway,Sampdoria,57,77,220000,...,55+2,57+2,Askildsen,Kristoffer,Kristoffer Askildsen,Kristoffer Askildsen,19,2500000.00,Central Midfield,Serie A
584,M. Caicedo,Moisés Isaac Caicedo Corozo,18,178,73,Ecuador,Independiente del Valle,56,71,180000,...,52+2,54+2,Caicedo,Moisés,Moisés Caicedo,Moisés Caicedo,19,5000000.00,Central Midfield,Premier League


388


,short_name,long_name,age_x,height_cm,weight_kg,nationality_name,club_name,overall,potential,value_eur,...,rcb,rb,last_name,first_name,full_name,name,age_y,fee,position,league
0,E. Hazard,Eden Hazard,28,175,74,Belgium,Real Madrid,91,91,90000000,...,49+3,61+3,Hazard,Eden,Eden Hazard,Eden Hazard,28,115000000.00,Left Winger,Laliga
2,A. Griezmann,Antoine Griezmann,28,176,73,France,FC Barcelona,89,89,69000000,...,65+3,70+3,Griezmann,Antoine,Antoine Griezmann,Antoine Griezmann,28,120000000.00,Second Striker,Laliga
4,C. Eriksen,Christian Dannemann Eriksen,27,181,76,Denmark,Tottenham Hotspur,88,89,68000000,...,57+3,66+3,Eriksen,Christian,Christian Eriksen,Christian Eriksen,27,27000000.00,Attacking Midfield,Premier League
6,M. Hummels,Mats Hummels,30,191,94,Germany,Borussia Dortmund,87,87,41000000,...,84+3,78+3,Hummels,Mats,Mats Hummels,Mats Hummels,30,30500000.00,Centre-Back,1 Bundesliga
8,K. Navas,Keylor Navas,32,185,80,Costa Rica,Real Madrid,87,87,30500000,...,NaN,NaN,Navas,Keylor,Keylor Navas,Keylor Navas,32,15000000.00,Goalkeeper,Laliga
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,S. McCallum,Sam McCallum,18,172,66,England,Coventry City,55,75,150000,...,50+2,53+2,McCallum,Sam,Sam McCallum,Sam McCallum,19,4150000.00,Left-Back,Premier League
905,M. Leverbe,Maxime Leverbe,22,188,76,France,Chievo Verona,55,63,110000,...,53+2,49+2,Leverbe,Maxime,Maxime Leverbe,Maxime Leverbe,22,600000.00,Centre-Back,Serie A
906,A. Karo,Andreas Karo,22,180,65,Cyprus,US Salernitana 1919,50,63,50000,...,49+2,44+2,Karo,Andreas,Andreas Karo,Andreas Karo,22,165000.00,Centre-Back,Serie A
907,J. Branthwaite,Jarrad Branthwaite,17,193,70,England,Carlisle United,49,64,50000,...,47+2,44+2,Branthwaite,Jarrad,Jarrad Branthwaite,Jarrad Branthwaite,17,1100000.00,Centre-Back,Premier League


602


In [ ]:
df_fifa_21 = df_fifa.copy()
df_fifa_21['last_name'] = df_fifa_21['short_name'].apply(extract_last_name)
df_fifa_21['first_name'] = df_fifa_21['long_name'].apply(extract_first_name)
df_fifa_21['full_name'] = df_fifa_21['first_name'] + ' ' + df['last_name']

print ("len before drop", len(df_fifa_21))
df_fifa_21= df_fifa_21[df_fifa_21['league_name'].isin(['Italian Serie A', 'German 1. Bundesliga', 'French Ligue 1', 'English Premier League', 'Spain Primera Division' ])]
df_fifa_21['league_name'].unique()
print ("len after drop", len(df_fifa_21))

len before drop 18944
len after drop 3092


In [ ]:
# for i in range df_2021_transfer

In [ ]:

# def fuzzy_merge(df1, df2, key1, key2, threshold=90, limit=1):
#     df1[key1] = df1[key1].astype(str)
#     df2[key2] = df2[key2].astype(str)

#     # Convert df2[key2] to a list for faster processing
#     choices = df2[key2].tolist()

#     # Use vectorized operations
#     tqdm.pandas(desc="Matching")
#     matches = df1[key1].progress_apply(lambda x: process.extractOne(x, choices, score_cutoff=threshold))

#     df1['match'] = matches.apply(lambda x: x[0] if x else None)
#     df1['match_score'] = matches.apply(lambda x: x[1] if x else None)

#     # Perform the actual merge
#     df_merged = pd.merge(df1, df2, left_on='match', right_on=key2, how='left')

#     # Drop the temporary 'match' column
#     df_merged.drop('match', axis=1, inplace=True)

#     return df_merged

# df_merged = fuzzy_merge(df, df_fifa, 'name', 'short_name', threshold=80)

In [ ]:
# df_merged = pd.merge(df, df_fifa, left_on='name')